In [1]:
import pandas as pd
import numpy as np
import os
import wget

In [2]:
filepath = os.path.join('.', 'data', 'raw')
os.makedirs(filepath)
wget.download('https://physionet.org/files/challenge-2012/1.0.0/set-a.zip', filepath)
wget.download('https://physionet.org/files/challenge-2012/1.0.0/Outcomes-a.txt', filepath)

os.chdir(filepath)
!unzip -u -q set-a.zip
os.chdir(os.path.join(os.pardir, os.pardir))

In [3]:
# join patient files into one dataframe
df = pd.DataFrame()
fp = os.path.join('.', 'data', 'raw', 'set-a')
for filename in os.listdir(fp):
    with open(os.path.join(fp, filename), 'r') as openfile:
        df_temp = pd.read_csv(openfile)
        df_temp['RecordID'] = df_temp[df_temp['Parameter']=='RecordID'].at[0,'Value'].astype(int) # create RecordID column
        df = pd.concat([df, df_temp], ignore_index=True)

# set negative values to null
df.loc[df['Value']<0, 'Value'] = np.nan

In [4]:
# create a separate dataframe w/ general descriptors
df_gen_desc = df.loc[df['Time']=='00:00', :].copy()
df_gen_desc = df_gen_desc.loc[df['Parameter'].isin(['Weight', 'Gender', 'Height', 'ICUType', 'Age'])]

# Get only the last value per parameter per patient, as some patients have multiple values for Age
df_gen_desc = df_gen_desc.groupby(['RecordID', 'Parameter'])[['Value']].last().reset_index()

# drop general descriptors from original dataframe, except for weight, which is recorded as a timeseries
df = df[~df['Parameter'].isin(['RecordID', 'Gender', 'Height', 'ICUType', 'Age'])]

# convert time from string to minutes elapsed
df['Time'] = df['Time'].map(lambda x: int(x.split(':')[0])*60 + int(x.split(':')[1]))

In [5]:
# save prepared data to disk
filepath = os.path.join('.', 'data', 'interim')
os.makedirs(filepath)
os.chdir(filepath)
df.to_csv('set-a_timeseries.csv', index=False)
df_gen_desc.to_csv('set-a_general-descriptors.csv', index=False)
os.chdir(os.path.join(os.pardir, os.pardir))